#### **Part 1 - Analyzing and preparing the charts:**
##### **1.1 - Importing packages and modules:**

In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import math
from prettytable import PrettyTable
import creds

##### **1.2 - Analyzing and cleaning the charts:**

In [ ]:
# Create dataframe from files in week_charts folder:

# Getting the files' names from \week_charts folder and creating a list for the files' analysis:
files_names = os.listdir('charts')
generic_problems = []
specific_problems = []

# Loop through the names and reading the .csv files:
for file in files_names:
    path = 'charts/' + str(file)
    chart = pd.read_csv(path)
    
    # Checking for generic problems with respect to charts (missing and duplicated rows):
    if (len(chart) != 200) or len(chart[chart.duplicated() == True]):
        name = str(file)
        missing_rows = '-----' if (200 - len(chart)) == 0 else (200 - len(chart))
        duplicated_rows = '-----' if len(chart[chart.duplicated() == True]) == 0 else len(chart[chart.duplicated() == True])
        generic_problem = [name, missing_rows, duplicated_rows]
        generic_problems.append(generic_problem)
        
    # Checking for specific problems with charts' columns:
    
    if (len(chart[chart['uri'].isnull()]) > 0 or len(chart[chart['uri'] == '']) > 0 or
        len(chart[chart['uri'].duplicated()]) > 0 or
        len(chart[chart['artist_names'].isnull()]) > 0 or len(chart[chart['artist_names'] == '']) > 0 or
        len(chart[chart['track_name'].isnull()]) > 0 or len(chart[chart['track_name'] == '']) > 0 or
        len(chart[chart['source'].isnull()]) > 0 or len(chart[chart['source'] == '']) > 0 or
        len(chart[chart['weeks_on_chart'].isnull()]) or
        min(chart['weeks_on_chart']) <= 0 or
        len(chart[chart['streams'].isnull()]) or
        min(chart['streams']) <= 0):

        name = str(file)
        missing_uri = ('-----' if (len(chart[chart['uri'].isnull()]) == 0 and len(chart[chart['uri'] == '']) == 0) 
                       else (len(chart[chart['uri'].isnull()]) + len(chart[chart['uri'] == ''])))                
        duplicated_uri = ('-----' if len(chart[chart['uri'].duplicated()]) == 0 else len(chart[chart['uri'].duplicated()]))        
        missing_artist_name = ('-----' if (len(chart[chart['artist_names'].isnull()]) == 0 and len(chart[chart['artist_names'] == '']) == 0) 
                       else (len(chart[chart['artist_names'].isnull()]) + len(chart[chart['artist_names'] == ''])))        
        missing_track_name = ('-----' if (len(chart[chart['track_name'].isnull()]) == 0 and len(chart[chart['track_name'] == '']) == 0) 
                       else (len(chart[chart['track_name'].isnull()]) + len(chart[chart['track_name'] == ''])))        
        missing_source = ('-----' if (len(chart[chart['source'].isnull()]) == 0 and len(chart[chart['source'] == '']) == 0) 
                       else (len(chart[chart['source'].isnull()]) + len(chart[chart['source'] == ''])))        
        missing_weeks_on_charts = ('-----' if (len(chart[chart['weeks_on_chart'].isnull()]) == 0) 
                       else (len(chart[chart['weeks_on_chart'].isnull()])))        
        inconsistent_weeks_on_chart = ('-----' if (len(chart[chart['weeks_on_chart'] <= 0]) <= 0)
                                       else (len(chart[chart['weeks_on_chart'] <= 1])))        
        missing_streams = ('-----' if (len(chart[chart['streams'].isnull()]) == 0) 
                       else (len(chart[chart['streams'].isnull()])))        
        inconsistent_streams = ('-----' if (len(chart[chart['streams'] <= 0]) <= 0)
                                else (len(chart[chart['streams'] <= 1])))
        
        specific_problem = [name, missing_uri, duplicated_uri, missing_artist_name, missing_track_name, duplicated_track_name,
                            missing_source, missing_weeks_on_charts, inconsistent_weeks_on_chart, missing_streams, inconsistent_streams]
        specific_problems.append(specific_problem)    
    
# Printing the generic problems with the charts:
if len(generic_problems) != 0:
    gen_pro_table = PrettyTable(['Files Names', 'Missing Rows', "Duplicated Rows"])
    gen_pro_table.title = "Files with Generic Problems"
    for problem in generic_problems:
        gen_pro_table.add_row(problem)
    print(gen_pro_table)
else:
    print('There are no generic problems (missing or dumplicated rows) with the charts downloaded.')
    
# Printing the specific problems with the charts:
if len(specific_problems) != 0:
    spe_pro_table = PrettyTable(['Files Names', 'Missing uri', 'Duplicated uri', 'Missing artist_names', 'Missing track_name',
                                 'Missing source', 'Missing weeks_on_chart', 'Inconsistent weeks_on_chart',
                                 'Missing streams', 'Inconsistent streams'])
    spe_pro_table.title = "Files with Specific Problems"
    for problem in specific_problems:
        spe_pro_table.add_row(problem)
    print(spe_pro_table)
else:
    print("\nThere are no charts with specific problems in its columns' values.")

##### **1.3 - Removing unwanted columns, joining the charts and organizing the dataframe:**

In [5]:
# Merging the charts in one single dataframe:

merged_charts_df = pd.DataFrame()

for file in files_names:
    path = 'charts/' + str(file)
    new_df = pd.read_csv(path)
    new_df = new_df.drop(['rank', 'peak_rank', 'previous_rank'], axis=1)
    merged_charts_df = pd.concat([merged_charts_df, new_df])

# Grouping the rows by 'uri', summing the streams and getting the max from 'weeks_on_chart' column:

merged_charts_df = merged_charts_df.groupby('uri', as_index=False).agg({'artist_names': 'first', 'track_name': 'first',
                                                                        'source': 'first', 'weeks_on_chart': 'max',
                                                                        'streams': 'sum'})

# Getting the tracks ids from 'uri' and renaming the column to 'id':

merged_charts_df['uri'] = merged_charts_df['uri'].str.replace('spotify:track:','')
merged_charts_df.rename(columns = {'uri':'id'}, inplace = True)

#### **Part 2 - Using Spotify's API to get the audio features of the tracks:**

##### **2.1 - Getting access to Spotify's API service:**

In [6]:
# Getting access to Spotify's API service:

# API information:
client_id = creds.client_id
client_secret = creds.client_secret

# Getting an access token:
AUTH_URL = 'https://accounts.spotify.com/api/token'
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})
auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

##### **2.2 - Using the tracks ids to request information about the audio features of the tracks:**

In [7]:
# Gathering the 'ids' to request more information from Spotify's API:

tracks_id = merged_charts_df['id'].values.tolist()

# Getting information about the tracks' audio features from Spotify's API service:

BASE_URL = 'https://api.spotify.com/v1/audio-features?ids='

responses = []

for i in range(math.ceil(len(tracks_id) / 100)):
    start_int = i*100
    end_int = start_int + 100
    ids = ",".join(tracks_id[(start_int):(end_int)])
    r = requests.get(BASE_URL + ids, headers=headers)
    responses.append(r.json())

##### **2.3 - Creating a dataframe to store the information from Spotify's API:**

In [8]:
# Gathering all the information from Spotify's API in a new dataframe:

list_of_features = []

for response in responses:
    res = list(next(iter(response.values())))
    for re in res:
        values  = list(re.values())
        list_of_features.append(values)
        
audio_features_df = pd.DataFrame(columns=re.keys())
for item in list_of_features:
    audio_features_df.loc[len(audio_features_df)] = item

#### **Part 3 - Joining the two dataframes (charts and audio features), organizing and creating the .csv file:**

##### **3.1 - Joining the two dataframes (charts and audio features)**

In [9]:
# Merging the new dataframe with the merged_charts_df:

top_songs_audio_features_df = pd.merge(merged_charts_df, audio_features_df, on='id')

##### **3.2 - Organizing the dataframe: converting values, removing and rearranging columns:**

In [10]:
# Removing unwanted information from the dataframe: 

top_songs_audio_features_df = top_songs_audio_features_df.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1)

# Rearranging the columns in accordance to the nature of the data:

top_songs_audio_features_df = top_songs_audio_features_df[['id', 'artist_names', 'track_name','source', 'key', 'mode', 
                                                           'time_signature', 'danceability', 'energy', 'speechiness', 'acousticness',
                                                           'instrumentalness', 'liveness', 'valence', 'loudness', 'tempo',
                                                           'duration_ms', 'weeks_on_chart', 'streams']]
# Convert keys' numbers to keys' names:
top_songs_audio_features_df['key'] = top_songs_audio_features_df['key'].replace([-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 
                                                                  ["No key detected", "C","C#/Db","D","D#/Eb",
                                                                   "E","F","F#/Gb","G","G#/Ab","A","A#/Bb","B"])

# Convert modes' numbers to modes' names:
top_songs_audio_features_df['mode'] = top_songs_audio_features_df['mode'].replace([0, 1], ['Minor', 'Major'])

# Convert time_signature number to names:
top_songs_audio_features_df['time_signature'] = (top_songs_audio_features_df['time_signature'].replace
                                                 ([3, 4, 5, 6, 7], ['3 beats', '4 beats', '5 beats', '6 beats', '7 beats']))

##### **3.3 - Creating the .csv file with all the information:**

In [11]:
# Convert the dataframe into a .csv file:

top_songs_audio_features_df.to_csv('spotify_top_songs_audio_features.csv', index=None)